# Resumo do Notebook



Este notebook é organizado em três seções principais:

1. **Vetores de Palavras Pré-Treinados**:
   - Carregue os vetores de palavras FastText de 300 dimensões e reduza-os para 100 dimensões. Esta redução otimiza o uso de espaço e melhora o desempenho do modelo sem perder muita informação contextual.

2. **Treinamento do Modelo**:
   - Treine e salve modelos XGBoost utilizando os vetores de 100 dimensões. Esta seção é subdividida em dois modelos distintos:
     - **XGBoost para Classificação Negativo vs Restante**:
       - Treine um modelo XGBoost para distinguir entre sentimentos negativos e todos os outros sentimentos (não-negativos).
     - **XGBoost para Classificação Positivo vs Neutro**:
       - Treine um modelo XGBoost para diferenciar entre sentimentos positivos e neutros, excluindo frases com sentimentos negativos.

3. **API: Processamento de Texto**:
   - Esta seção integra todos os processos anteriores para analisar e classificar uma frase individual. Inclui as etapas de pré-processamento, vetorização (utilizando os vetores de 100 dimensões) e classificação. A API final permite determinar se a frase é negativa, positiva ou neutra, aplicando o pipeline de classificação desenvolvido.

# Instruções para o uso do Notebook

Para rodar o notebook a seguir corretamente, é necessário seguir alguns passos prévios. Primeiro, baixe os datasets disponíveis no link a seguir: [Datasets](https://drive.google.com/drive/folders/1bm6iQenyZ63gbw_s7j0md8UaHP84O0Vn?usp=sharing) (não é necessário todos, a princípio, somente os citados abaixo). Esses datasets não estão incluídos no notebook devido ao seu tamanho, por isso são ignorados pelo arquivo `.gitignore`. Todos eles devem ser colocados na pasta [`data`](data), localizada em `src/Notebook/Api_Sprint_4/data`.

Os arquivos necessários são:

- **cc.en.300.bin**: [Download](https://drive.google.com/file/d/1eM6TfyZIUt6YlCOus5C8nCU1UkmqRu2V/view?usp=sharing) - arquivo bin com palavras pré-treinadas para o FastText.
- **tweets_uber.csv**: [Baixar Arquivo](https://drive.google.com/file/d/1cot0O9YoNDQa6bPpVgboRhMIOOsoh2rI/view?usp=sharing) - Este arquivo contém uma coleção de tweets relacionados à Uber. Vale destacar que este dataset não é o original fornecido pela *Uber*, pois foi modificado para fins de análise.

Todos são necessários para rodar as células seguintes.

Outras recomendações:

- Mantenha o C++ do seu computador atualizado, pois determinadas bibliotecas são compiladas com essa linguagem.
- Caso queira conferir as versões mais atualizaas das bibliotecas, elas estão detalhadas no aquivo [``requirements.txt``](../../requirements.txt). Para usar este arquivo, basta rodar no terminal o comando: ``pip install -r requirements.txt``. Porém, a princípio, é possível ter acesso a todas as bilioteca ao rodar a célula de "Instalação e Importação das Bibliotecas" deste Notebook.

As funções abaixo adaptam e integram as etapas de pré-processamento de texto, vetorização utilizando FastText, e qualificação do modelo através do XGBoost.

# Importação

In [ ]:
# Instalação de "Bibliotecas Mágicas"
%pip install pandas
%pip install numpy
%pip install nltk
%pip install emoji
%pip install wordcloud
%pip install emot
%pip install imbalanced-learn
%pip install matplotlib
%pip install scikit-learn
%pip install spacy
!python -m spacy download en_core_web_md
%pip install scipy==1.11
%pip install gensim
%pip install scikit-learn
%pip install seaborn
%pip install xgboost
%pip install --upgrade scikit-learn
%pip install --upgrade imbalanced-learn
%pip install fasttext
%pip install fasttext-wheel

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
import nltk
import re
import emoji
import os
import pickle
import fasttext
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from imblearn.over_sampling import SMOTE

# Certifique-se de ter baixado os pacotes necessários do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Vetores de Palavras Pré-Treinadas

Para otimizar o desempenho e a eficiência do modelo, o arquivo de vetores de palavras pré-treinadas fornecido pelo Facebook foi modificado. Originalmente, os vetores possuem 300 dimensões. Utilizando a vetorização FastText, reduzimos esses vetores para apenas 100 dimensões. Essa transformação reduz significativamente a complexidade computacional e o espaço de armazenamento, mantendo, ao mesmo tempo, uma representação adequada das palavras para o treinamento do modelo.

In [ ]:
def reduce_fasttext_dimensions(input_model_path, output_file_path, num_dimensions=100):
    """
    Reduz as dimensões dos vetores de um modelo fastText, mantendo apenas as primeiras num_dimensions dimensões,
    e salva o resultado em um arquivo de texto no formato .vec.

    Args:
    input_model_path (str): Caminho para o modelo fastText original com 300 dimensões.
    output_file_path (str): Caminho para salvar o novo arquivo com dimensões reduzidas.
    num_dimensions (int): Número de dimensões a manter. O padrão é 100.

    Returns:
    None
    """
    # Carregar o modelo fastText completo
    model = fasttext.load_model(input_model_path)
    
    words = model.get_words()
    
    # Abrir o arquivo para escrever os vetores reduzidos com codificação UTF-8
    with open(output_file_path, 'w', encoding='utf-8') as f:
        # Escrever a linha de cabeçalho com o número de palavras e dimensões
        f.write(f"{len(words)} {num_dimensions}\n")
        
        for word in words:
            # Obter o vetor do modelo original
            original_vector = model.get_word_vector(word)
            
            # Manter apenas as primeiras num_dimensions dimensões
            reduced_vector = original_vector[:num_dimensions]
            
            # Converter o vetor reduzido para uma string de valores
            vector_str = ' '.join(map(str, reduced_vector))
            
            # Escrever a palavra e o vetor reduzido no arquivo
            f.write(f"{word} {vector_str}\n")



In [ ]:
# Exemplo de uso
input_model_path = './data/cc.en.300.bin'  # Caminho para o modelo fastText original com 300 dimensões
output_file_path = './data/cc.en.100.vec'  # Caminho para salvar o novo arquivo com dimensões reduzidas

reduce_fasttext_dimensions(input_model_path, output_file_path, num_dimensions=100)

print(f"Novo arquivo com vetores reduzidos salvo em {output_file_path}")

In [ ]:
def process_and_vectorize_csv(csv_path, vec_path, output_path):
    """
    Carrega os embeddings do fastText a partir de um arquivo .vec, vetoriza a coluna 'processed_text' de um CSV,
    e salva o resultado em um novo CSV.

    Args:
    csv_path (str): Caminho para o arquivo CSV com os dados de entrada.
    vec_path (str): Caminho para o arquivo .vec com os embeddings.
    output_path (str): Caminho para salvar o arquivo CSV com os resultados.
    """
    # Verificar se os arquivos existem
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Arquivo CSV não encontrado no caminho: {csv_path}")
    if not os.path.exists(vec_path):
        raise FileNotFoundError(f"Arquivo de embeddings não encontrado no caminho: {vec_path}")
    
    # Carregar os embeddings do arquivo .vec
    print("Carregando os embeddings...")
    embeddings = {}
    with open(vec_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector

    # Função para converter texto em vetor
    def text_to_vector(text, embeddings):
        words = text.split()
        word_vectors = [embeddings[word] for word in words if word in embeddings]
        if len(word_vectors) == 0:
            return np.zeros(len(next(iter(embeddings.values()))))  # Tamanho do vetor de embeddings
        return np.mean(word_vectors, axis=0)
    
    # Carregar o arquivo CSV
    print("Carregando o arquivo CSV...")
    df = pd.read_csv(csv_path)
    
    # Vetorizar a coluna 'processed_text'
    print("Vetorizar a coluna 'processed_text'...")
    vectors = df['processed_text'].apply(lambda text: text_to_vector(text, embeddings))
    
    # Converter os vetores para um DataFrame
    vectors_df = pd.DataFrame(vectors.tolist(), index=df.index)
    
    # Combinar com a coluna 'sentiment'
    result_df = pd.concat([df['sentiment'], vectors_df], axis=1)
    
    # Salvar o DataFrame resultante em um novo arquivo CSV
    print("Salvando o arquivo resultante...")
    result_df.to_csv(output_path, index=False)
    print(f"Vetorizações salvas em: {output_path}")

In [ ]:
csv_path = './data/tweets_uber.csv'
vec_path = './data/cc.en.100.vec'
output_path = './data/tweets_uber_vectorized.csv'

process_and_vectorize_csv(csv_path, vec_path, output_path)

# Treinamento do Modelo

O código do modelo foi adaptado a partir do arquivo [Model_improvements](../Api_Sprint_4/Model_improvements.ipynb) para realizar as seguintes melhorias:

- **Redução do Conjunto de Treinamento**: O treinamento agora utiliza apenas 100 vetores, permitindo uma execução mais rápida e eficiente para fins de testes.
- **Suporte a Arquivos .vec**: O modelo foi ajustado para aceitar arquivos de vetor no formato `.vec` ao invés de `.bin`, oferecendo maior flexibilidade no uso de diferentes fontes de dados.
- **Persistência do Modelo**: Implementação de uma funcionalidade para salvar o modelo treinado, facilitando o armazenamento e a reutilização do modelo em futuras previsões.

In [ ]:
def train_xgboost_model_negative_vs_rest(data_for_training_path, model_save_path):
    """
    Treina um modelo XGBoost para classificação binária (negativa ou não negativa) e salva o modelo treinado em um arquivo .pkl.

    Inputs:
        data_for_training_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.
        model_save_path (str): Caminho para salvar o modelo treinado como um arquivo .pkl.

    Outputs:
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados de treinamento a partir de um arquivo CSV
        data_for_training = pd.read_csv(data_for_training_path)

        if 'id' in data_for_training.columns:
            data_for_training = data_for_training.drop(columns=['id'])

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data_for_training.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Dividir os dados em features (X) e target (y)
        X = data_for_training.drop('sentiment', axis=1)
        y = data_for_training['sentiment']

        # Reclassificar as classes para binário
        y = y.apply(lambda x: 1 if x == -1 else 0)

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Aplicar SMOTE para balancear as classes no conjunto de treinamento
        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        # Configurar o modelo XGBoost
        model = xgb.XGBClassifier(max_depth=6, eta=0.3, objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

        # Realizar validação cruzada
        cv_scores = cross_val_score(model, X_train_res, y_train_res, cv=5, scoring='f1_weighted')

        # Treinar o modelo
        model.fit(X_train_res, y_train_res)

        # Fazer previsões
        y_pred = model.predict(X_test)

        # Verificar os rótulos únicos nas previsões
        print("Rótulos únicos nas previsões:", np.unique(y_pred))

        # Calcular e exibir o relatório de classificação
        class_report = classification_report(y_test, y_pred)
        print("Classification Report:\n", class_report)

        # Calcular métricas
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'f1_score': f1_score(y_test, y_pred, average='weighted'),
            'kappa': cohen_kappa_score(y_test, y_pred),
            'mean_cross_val_f1': cv_scores.mean(),
            'std_cross_val_f1': cv_scores.std()
        }

        print(metrics)

        # Salvar o modelo treinado em um arquivo .pkl
        with open(model_save_path, 'wb') as file:
            pickle.dump(model, file)

        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None


In [ ]:
# Chamada da Função:
metrics = train_xgboost_model_negative_vs_rest('./data/tweets_uber_vectorized.csv', './models/xgboost_negative_vs_rest.pkl')

In [ ]:
def train_xgboost_model_positive_vs_neutral(file_path, model_save_path):
    """
    Treina um modelo XGBoost para classificação binária (neutra ou positiva), excluindo frases negativas,
    e salva o modelo treinado em um arquivo .pkl.

    Inputs:
        file_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.
        model_save_path (str): Caminho para salvar o modelo treinado como um arquivo .pkl.

    Outputs:
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados
        data = pd.read_csv(file_path)

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Filtrar para remover frases negativas
        data = data[data['sentiment'] != -1]

        # Dividir os dados em features (X) e target (y)
        X = data.drop('sentiment', axis=1)
        y = data['sentiment']

        # Reclassificar as classes para binário
        y = y.apply(lambda x: 1 if x == 1 else 0)

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Aplicar SMOTE para balancear as classes no conjunto de treinamento
        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        # Configurar o modelo XGBoost
        model = xgb.XGBClassifier(max_depth=6, eta=0.3, objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

        # Realizar validação cruzada
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train_res, y_train_res, cv=cv, scoring='f1_weighted')

        # Treinar o modelo
        model.fit(X_train_res, y_train_res)

        # Fazer previsões
        y_pred = model.predict(X_test)

        # Verificar os rótulos únicos nas previsões
        print("Rótulos únicos nas previsões:", np.unique(y_pred))

        # Calcular e exibir o relatório de classificação
        class_report = classification_report(y_test, y_pred)
        print("Classification Report:\n", class_report)

        # Calcular métricas
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'f1_score': f1_score(y_test, y_pred, average='weighted'),
            'kappa': cohen_kappa_score(y_test, y_pred),
            'mean_cross_val_f1': cv_scores.mean(),
            'std_cross_val_f1': cv_scores.std()
        }

        print(metrics)

        # Salvar o modelo treinado em um arquivo .pkl
        with open(model_save_path, 'wb') as file:
            pickle.dump(model, file)

        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

In [ ]:
# Chamada da função:
metrics = train_xgboost_model_positive_vs_neutral('./data/tweets_uber_vectorized.csv', './models/xgboost_positive_vs_neutral.pkl')

# API: Processamento de Texto

Os códigos de pré-processamento, vetorização e aplicação do modelo foram reorganizados e otimizados para processar uma única frase por vez. Esta abordagem simplificada permite que a API:

- **Pré-Processamento**: Limpar e preparar a frase para vetorização, removendo ruídos e aplicando técnicas de normalização.
- **Vetorização**: Converter a frase em vetores utilizando a vetorização FastText adaptada, agora com 100 dimensões, garantindo uma representação compacta e eficaz.
- **Modelo**: Aplicar o modelo treinado à frase vetorizada, fornecendo resultados rápidos e precisos para análises de sentimento ou outras classificações.

## Funções

### Pré-processamento


In [ ]:
# Função para converter emojis em palavras
def convert_emojis(text):
    return emoji.demojize(text)

# Função para limpar o texto e substituir apóstrofos por espaços
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<[^>]+>', '', text)  # Remove tags HTML
    text = re.sub(r'\'', ' ', text)  # Substitui apóstrofos por espaços
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove caracteres especiais e números, exceto espaços
    text = re.sub(r'\s+', ' ', text).strip()  # Corrige espaços extras
    return text

# Função para tokenizar o texto e converter para minúsculas
def tokenize(text):
    tokens = word_tokenize(text.lower())
    return tokens

# Função para remover stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

# Função para aplicar lematização
def apply_lemmatization(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

# Lista de palavras a serem mantidas
words_to_keep = ['uber']

# Função para corrigir a ortografia, mantendo palavras específicas
def correct_spelling(text):
    for i, word in enumerate(words_to_keep):
        text = text.replace(word, f'PLACEHOLDER_{i}')
    corrected_text = str(TextBlob(text).correct())
    for i, word in enumerate(words_to_keep):
        corrected_text = corrected_text.replace(f'PLACEHOLDER_{i}', word)
    return corrected_text

# Função principal de pré-processamento para uma única frase
def preprocess_text(text):
    text = text.lower()  # Converte para minúsculas
    text = convert_emojis(text)
    text = clean_text(text)
    text = correct_spelling(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    lemmatized_tokens = apply_lemmatization(tokens)
    return ' '.join(lemmatized_tokens)  # Retorna a frase lematizada


In [ ]:
# Exemplo de uso
example_text = "I love Uber's service! 🚗 #Awesome"
processed_text = preprocess_text(example_text)
print(processed_text)

### Vetorização

In [ ]:
def text_to_vector(sentence, vec_path):
    """
    Carrega os embeddings do fastText a partir de um arquivo .vec e converte uma frase em um vetor de embeddings.

    Args:
    sentence (str): Frase de entrada.
    vec_path (str): Caminho para o arquivo .vec com os embeddings.

    Returns:
    np.array: Vetor que representa a frase.
    """
    # Verifica se o arquivo .vec existe
    if not os.path.exists(vec_path):
        raise FileNotFoundError(f"Arquivo de embeddings não encontrado no caminho: {vec_path}")
    
    # Carrega os embeddings do arquivo .vec
    embeddings = {}
    with open(vec_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    
    # Converte a frase em um vetor usando a média dos embeddings das palavras
    words = sentence.split()
    word_vectors = [embeddings[word] for word in words if word in embeddings]
    
    if len(word_vectors) == 0:
        # Retorna um vetor de zeros se nenhuma palavra da frase estiver nos embeddings
        return np.zeros(len(next(iter(embeddings.values()))))
    
    # Calcula a média dos vetores das palavras
    return np.mean(word_vectors, axis=0)


In [ ]:
# Exemplo de uso:
vec_path = './data/cc.en.100.vec'
sentence = "love using uber automobile visit"
vector = text_to_vector(sentence, vec_path)

print("Vetor da frase:", vector)

### Função Negativo vs Positivos e Neutros

In [ ]:
# Função para classificar o texto como negativo vs resto
def classify_text_neg_rest(vectorized_text, model_file):
    """
    Classifica uma frase como negativa (0) ou não negativa (1) usando um modelo XGBoost treinado.

    Inputs:
        vectorized_text (np.ndarray): Vetor representando a frase vetorizada.
        model_file (str): Caminho para o arquivo pickle contendo o modelo XGBoost treinado.

    Output: int: Classificação da frase (0: negativa, 1: não negativa).
    """
    # Carregar o modelo XGBoost treinado
    with open(model_file, 'rb') as file:
        model = pickle.load(file)

    # Verificar se o vetor é unidimensional e transformar em um array 2D se necessário
    if len(vectorized_text.shape) == 1:
        vectorized_text = vectorized_text.reshape(1, -1)

    # Fazer a predição usando o modelo carregado
    prediction = model.predict(vectorized_text)
    
    return int(prediction[0])

### Função Positivos vs Neutros

In [ ]:
# Função para classificar o texto como positivo vs neutro
def classify_text_pos_neutral(vectorized_text, model_file):
    """
    Classifica uma frase como positiva (2) ou neutra (1) usando um modelo XGBoost treinado.

    Inputs:
        vectorized_text (np.ndarray): Vetor representando a frase vetorizada.
        model_file (str): Caminho para o arquivo pickle contendo o modelo XGBoost treinado.

    Output: int: Classificação da frase (2: positiva, 1: neutra).
    """
    # Carregar o modelo XGBoost treinado
    with open(model_file, 'rb') as file:
        model = pickle.load(file)

    # Verificar se o vetor é unidimensional e transformar em um array 2D se necessário
    if len(vectorized_text.shape) == 1:
        vectorized_text = vectorized_text.reshape(1, -1)

    # Fazer a predição usando o modelo carregado
    prediction = model.predict(vectorized_text)
    
    return int(prediction[0])

## Junção das funções

In [ ]:
def process_and_classify_text(text, fasttext_model_path, model_neg_vs_rest_path, model_pos_vs_neutral_path):
    # Pré-processar o texto
    preprocessed_text = preprocess_text(text)
    
    # Vetorizar o texto
    vectorized_text = text_to_vector(preprocessed_text, fasttext_model_path)
    
    # Classificar o texto como negativo vs resto
    neg_vs_rest = classify_text_neg_rest(vectorized_text, model_neg_vs_rest_path)
    
    if neg_vs_rest == 0:  # Se não for negativo, classificar entre positivo e neutro
        pos_vs_neutral = classify_text_pos_neutral(vectorized_text, model_pos_vs_neutral_path)
        return 1 if pos_vs_neutral == 1 else 0  # 1: positivo, 0: neutro
    else:
        return -1  # -1: negativo

In [ ]:
# Exemplo de uso
fasttext_model_path = './data/cc.en.100.vec'
model_neg_vs_rest_path = './models/xgboost_negative_vs_rest.pkl'
model_pos_vs_neutral_path = './models/xgboost_positive_vs_neutral.pkl'

text = "I love Uber's service! 🚗 #Awesome"

result = process_and_classify_text(text, fasttext_model_path, model_neg_vs_rest_path, model_pos_vs_neutral_path)
print(f"Classificação: {result} (-1: Negativo, 0: Neutro, 1: Positivo)")